In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from sch.models import *
from sch.actions import *

In [3]:
slot = Slot.objects.get(slug__contains='2022-11-05-RS')
def resolveTurnaroundWithinDay (turnaroundSlot):
        emp = turnaroundSlot.employee
        if emp.evening_pref:
            wd = turnaroundSlot.workday
            sfta = turnaroundSlot.shift
        elif emp.evening_pref == False:
            wd = turnaroundSlot.workday.prevWD()
            sfta = Slot.objects.get(workday=wd,employee=emp)
        
        sfts = emp.shifts_available.all()
        print('shifts_available',sfts)
        noIncompatibles = []
        for sft in sfts:
            if not Slot.objects.incompatible_slots(workday=wd,shift=sft).filter(employee=emp).exists():
                noIncompatibles += [sft]
                print(sft, noIncompatibles)
        List = []
        for sft in noIncompatibles:
            checking = Slot.objects.filter(workday=wd,shift=sft)
            print(checking)
            if checking.exists():
                if sfta in checking.first().employee.shifts_available.all():
                    if not Slot.objects.incompatible_slots(workday=wd,shift=sfta).filter(employee=checking.first().employee).exists():
                        List += [sfta]  
        print(List)  
        if len(List) == 1:
            return List[0]  
        return List[random.randint(0,len(List)-1)]

In [6]:
vals = []
for wd in Workday.objects.filter(iweekday=1): #type: Workday
    vals += [int(wd.percFilled * 100)]
print(vals)
print (sum(vals)/len(vals))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 100, 100, 100, 100, 100, 0, 0, 0, 0, 0]
11.538461538461538


In [7]:
schedule7 = Workday.objects.filter(ischedule=7)
percFillVals = []
for wd in schedule7:
    percFillVals += [int(wd.percFilled*100)]
print(sum(percFillVals)/ len(percFillVals),"%")
        

96.33333333333333 %


In [8]:
wds = Workday.objects.filter(date__year=2022,iweek=3).first()
wds.who_can_fill(Shift.objects.get(name='MI'))


<EmployeeManager [<Employee: Josh>, <Employee: Sabrina>, <Employee: Brianna>, <Employee: Esperanza>, <Employee: Elisia>, <Employee: Jaden>, <Employee: Trisha>, <Employee: Tiffany>, <Employee: David>, <Employee: Nicki>, <Employee: Leslie>]>